## Graph

In [1]:
from utils import *

adj, feat, labels, idx_train, idx_val, idx_test, ano_labels = load('cora')
print("adj: ", adj.shape)
print("features: ", feat.shape)
print("train_size:", len(idx_train))
print("val_size:", len(idx_val))
print("test_size:", len(idx_test))

adj:  (2708, 2708)
features:  (2708, 1433)


In [33]:
# =============== Dominant ===============
def load_anomaly_detection_dataset(dataset, datadir='data'):
    data_mat = sio.loadmat(f'{datadir}/{dataset}.mat')
    adj = data_mat['Network']
    feat = data_mat['Attributes']
    truth = data_mat['Label']
    truth = truth.flatten()

    adj_norm = normalize_adj(adj + sp.eye(adj.shape[0]))
    adj_norm = adj_norm.toarray()
    adj = adj + sp.eye(adj.shape[0])
    adj = adj.toarray()
    feat = feat.toarray()
    return adj_norm, feat, truth, adj

adj, attrs, label, adj_label = load_anomaly_detection_dataset('ACM')
print("normalized adj: ", adj.shape)
print("features: ", attrs.shape)
print("label: ", label.shape)
print("original adj: ", adj_label.shape)
print("Sum or anomalies: ", sum(label))

normalized adj:  (16484, 16484)
features:  (16484, 8337)
label:  (16484,)
original adj:  (16484, 16484)
Sum or anomalies:  597


### MAE: random_masking

In [148]:
x = torch.rand((4, 5, 2))
x

tensor([[[0.4242, 0.2900],
         [0.3482, 0.5577],
         [0.2255, 0.1098],
         [0.8989, 0.5212],
         [0.7968, 0.0127]],

        [[0.3815, 0.1038],
         [0.2913, 0.9885],
         [0.6024, 0.2209],
         [0.1044, 0.9607],
         [0.6929, 0.2405]],

        [[0.1717, 0.6199],
         [0.4443, 0.0392],
         [0.0725, 0.5095],
         [0.2704, 0.8845],
         [0.1892, 0.1595]],

        [[0.6444, 0.4932],
         [0.7901, 0.1293],
         [0.1907, 0.7914],
         [0.7692, 0.4905],
         [0.1445, 0.6353]]])

In [62]:
noise = torch.rand(4, 5)
noise

tensor([[0.0835, 0.5561, 0.9291, 0.7978, 0.9702],
        [0.5953, 0.2089, 0.5204, 0.3862, 0.4703],
        [0.1807, 0.2943, 0.8598, 0.4118, 0.7235],
        [0.1801, 0.3224, 0.7655, 0.5698, 0.9442]])

In [63]:
ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
ids_shuffle

tensor([[0, 1, 3, 2, 4],
        [1, 3, 4, 2, 0],
        [0, 1, 3, 4, 2],
        [0, 1, 3, 2, 4]])

In [64]:
ids_restore = torch.argsort(ids_shuffle, dim=1)
ids_restore

tensor([[0, 1, 3, 2, 4],
        [4, 0, 3, 1, 2],
        [0, 1, 4, 2, 3],
        [0, 1, 3, 2, 4]])

In [65]:
ids_keep = ids_shuffle[:, :3]
ids_keep

tensor([[0, 1, 3],
        [1, 3, 4],
        [0, 1, 3],
        [0, 1, 3]])

In [66]:
x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, 2))
x_masked

tensor([[[0.0841, 0.7777],
         [0.2600, 0.6878],
         [0.5623, 0.3064]],

        [[0.5746, 0.9682],
         [0.3447, 0.8661],
         [0.1676, 0.4896]],

        [[0.4387, 0.9735],
         [0.8970, 0.7504],
         [0.7708, 0.4949]],

        [[0.0260, 0.9256],
         [0.9354, 0.9779],
         [0.6392, 0.9462]]])

In [67]:
mask = torch.ones([4, 5], device=x.device)
mask[:, :3] = 0
# unshuffle to get the binary mask
mask = torch.gather(mask, dim=1, index=ids_restore)
mask

tensor([[0., 0., 1., 0., 1.],
        [1., 0., 1., 0., 0.],
        [0., 0., 1., 0., 1.],
        [0., 0., 1., 0., 1.]])

### Debug:

In [1]:
import moco as moco

print(moco.__dict__['moco_graph_encoder'])

<function moco_graph_encoder at 0x7f81808a4790>


In [13]:
h_pl = torch.rand((4, 2))
c = torch.rand((4, 2))
print(h_pl)
print(c)

tensor([[0.0808, 0.9130],
        [0.8778, 0.8021],
        [0.0944, 0.6027],
        [0.2141, 0.1238]])
tensor([[0.8555, 0.2620],
        [0.9656, 0.1462],
        [0.1758, 0.5150],
        [0.4018, 0.3649]])


In [14]:
import torch.nn as nn

n_h = 2
f_k = nn.Bilinear(n_h, n_h, 1)

scs = []
scs.append(f_k(h_pl, c))
scs

[tensor([[ 0.3055],
         [ 0.4926],
         [-0.1603],
         [-0.1570]], grad_fn=<AddBackward0>)]

In [15]:
c_mi = c
negsamp_round = 1
for _ in range(negsamp_round):
    c_mi = torch.cat((c_mi[-2:-1,:], c_mi[:-1,:]),0)
    print(c_mi)
    scs.append(f_k(h_pl, c_mi))

tensor([[0.1758, 0.5150],
        [0.8555, 0.2620],
        [0.9656, 0.1462],
        [0.1758, 0.5150]])


In [17]:
print(scs)

[tensor([[ 0.3055],
        [ 0.4926],
        [-0.1603],
        [-0.1570]], grad_fn=<AddBackward0>), tensor([[-0.1441],
        [ 0.3266],
        [ 0.2137],
        [-0.2137]], grad_fn=<AddBackward0>)]


In [16]:
logits = torch.cat(tuple(scs))
logits

tensor([[ 0.3055],
        [ 0.4926],
        [-0.1603],
        [-0.1570],
        [-0.1441],
        [ 0.3266],
        [ 0.2137],
        [-0.2137]], grad_fn=<CatBackward0>)

In [18]:
x = torch.rand((4, 5, 2))
x

tensor([[[0.2789, 0.2368],
         [0.8589, 0.8995],
         [0.1355, 0.6173],
         [0.6932, 0.8482],
         [0.6499, 0.5748]],

        [[0.4149, 0.4989],
         [0.7000, 0.2095],
         [0.5814, 0.5955],
         [0.2664, 0.6382],
         [0.3337, 0.7935]],

        [[0.2979, 0.6018],
         [0.3499, 0.3546],
         [0.2195, 0.0341],
         [0.2673, 0.9819],
         [0.3262, 0.5714]],

        [[0.9486, 0.0165],
         [0.0677, 0.1777],
         [0.0727, 0.1079],
         [0.3486, 0.1212],
         [0.0447, 0.4711]]])

In [19]:
x[:, 0, :]

tensor([[0.2789, 0.2368],
        [0.4149, 0.4989],
        [0.2979, 0.6018],
        [0.9486, 0.0165]])

In [57]:
from dgl.nn.pytorch.glob import AvgPooling

avg = AvgPooling()

out = avg(temp, temp.ndata["features"])
print(out.shape)

torch.Size([1, 1433])


In [58]:
out = out.to(torch.float32)

In [59]:
drop = nn.Dropout(0.2)
linear = nn.Linear(1433, 32)
out = drop(linear(out))
print(out.shape)

torch.Size([1, 32])


In [60]:
score_over_layer = 0
score_over_layer += out

In [62]:
score_over_layer.shape

torch.Size([1, 32])

In [91]:
import torch

# Sinkhorn-Knopp
def sinkhorn(scores, eps=0.05, niters=3):
    Q = torch.exp(scores / eps).T
    Q /= sum(Q)
    K, B = Q.shape
    u, r, c = torch.zeros(K), torch.ones(K) / K, torch.ones(B) / B
    for _ in range(niters):
        u = torch.sum(Q, dim=1)
        Q *= (r / u).unsqueeze(1)
        Q *= (c / torch.sum(Q, dim=0)).unsqueeze(0)
    return (Q / torch.sum(Q, dim=0, keepdim=True)).T

z = torch.randn((4, 2))
print(z)

tensor([[-0.4061, -0.9381],
        [-0.6663,  0.9505],
        [ 0.6383,  1.0689],
        [ 1.8522, -0.2765]])


In [93]:
x = sinkhorn(z)
x

tensor([[9.9998e-01, 2.3970e-05],
        [9.0603e-15, 1.0000e+00],
        [1.8182e-04, 9.9982e-01],
        [1.0000e+00, 3.2391e-19]])

In [150]:
a = torch.rand((1, 4))
print(a)
b = F.softmax(a, dim=1)
print(b)

tensor([[0.8667, 0.7765, 0.2961, 0.1600]])
tensor([[0.3365, 0.3074, 0.1902, 0.1660]])


In [151]:
T = 0.8
numerator = np.exp(a) / T
print(numerator)

tensor([[2.9739, 2.7173, 1.6808, 1.4670]])


### MCL

In [132]:
import torch

y = torch.randint(0, 10, (4, ), dtype=torch.float32)
print(y)

tensor([8., 2., 2., 7.])


In [133]:
c = [y] * len(y)
mask = torch.stack(c)
print(mask.shape)

torch.Size([4, 4])


In [134]:
print(mask)

tensor([[8., 2., 2., 7.],
        [8., 2., 2., 7.],
        [8., 2., 2., 7.],
        [8., 2., 2., 7.]])


In [135]:
for i in range(len(mask)):
    mask[i] = mask[i]- mask[i][i]
    mask[i][i] -= 1
print(mask)
same_class_mask = (mask == 0).type(torch.uint8)
print(same_class_mask)

tensor([[-1., -6., -6., -1.],
        [ 6., -1.,  0.,  5.],
        [ 6.,  0., -1.,  5.],
        [ 1., -5., -5., -1.]])
tensor([[0, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 0]], dtype=torch.uint8)


In [140]:
random_choice = torch.nonzero(torch.tensor([0, 1, 1, 1])).squeeze()
random_idx = torch.randint(random_choice.size()[0], (1,))
random_idx

tensor([0])

In [142]:
label = torch.randint(0, 10, (4, ))
print(label)
label = label * 2
print(label)
label_copy1 = torch.unsqueeze(label, 1)
label_copy2 = torch.unsqueeze(label, 1)
concatted = torch.cat([label_copy1, label_copy2], 1)
print(concatted)
result = concatted.view([-1, 4*2]).squeeze()
print(result)
result[1::2]+=1
print(result)

tensor([2, 8, 6, 2])
tensor([ 4, 16, 12,  4])
tensor([[ 4,  4],
        [16, 16],
        [12, 12],
        [ 4,  4]])
tensor([ 4,  4, 16, 16, 12, 12,  4,  4])
tensor([ 4,  5, 16, 17, 12, 13,  4,  5])


In [147]:
bs = 2
sim_matrix = torch.randn((2*bs, 2*bs))
print(sim_matrix)
# mask diag
mask = (torch.ones_like(sim_matrix) - torch.eye(2 * bs, device=sim_matrix.device)).bool()
sim_matrix = sim_matrix.masked_select(mask).view(2 * bs, -1)
print(sim_matrix)

tensor([[-0.5663,  0.5757, -0.3176, -0.8358],
        [-0.2623,  0.6032,  1.3866,  0.5395],
        [ 0.1110, -1.1922, -1.3357,  0.4411],
        [ 0.4062, -1.6155,  0.3412, -1.2924]])
tensor([[ 0.5757, -0.3176, -0.8358],
        [-0.2623,  1.3866,  0.5395],
        [ 0.1110, -1.1922,  0.4411],
        [ 0.4062, -1.6155,  0.3412]])


In [169]:
import numpy as np

reps = []
for i in range(4):
    p = torch.randint(0, 10, (1, 4), dtype=torch.float32)
    p = F.normalize(p, dim=1)
    p = p.numpy()
    reps.append(p.squeeze())
print(reps)
cov = np.cov(reps, rowvar= False)
print(cov)

[array([0.4656903 , 0.54330534, 0.69853544, 0.        ], dtype=float32), array([0.3810004 , 0.        , 0.25400025, 0.8890009 ], dtype=float32), array([0.22222222, 0.5555556 , 0.6666667 , 0.44444445], dtype=float32), array([0.6620847 , 0.08276059, 0.        , 0.7448453 ], dtype=float32)]
[[ 0.03355981 -0.02669422 -0.04349511  0.01041558]
 [-0.02669422  0.08720489  0.09052245 -0.1017936 ]
 [-0.04349511  0.09052245  0.11381943 -0.10639684]
 [ 0.01041558 -0.1017936  -0.10639684  0.15427499]]


In [170]:
for i in range(4):
    x = int((i-1)/2)
    print(i)

0
1
2
3


In [154]:
targets = torch.arange(8)
print(targets)
targets[::2] += 1
print(targets)
targets[1::2] -= 1
print(targets)

tensor([0, 1, 2, 3, 4, 5, 6, 7])
tensor([1, 1, 3, 3, 5, 5, 7, 7])
tensor([1, 0, 3, 2, 5, 4, 7, 6])


In [110]:
import torch.nn.functional as F

diag = torch.eye(len(y))
mask += diag
mask_sum = mask.view(1,1,len(y),len(y))
mask_final = F.interpolate(mask_sum,scale_factor = 2,mode='nearest')
mask_final

tensor([[[[3., 3., 3., 3., 5., 5., 6., 6.],
          [3., 3., 3., 3., 5., 5., 6., 6.],
          [2., 2., 4., 4., 5., 5., 6., 6.],
          [2., 2., 4., 4., 5., 5., 6., 6.],
          [2., 2., 3., 3., 6., 6., 6., 6.],
          [2., 2., 3., 3., 6., 6., 6., 6.],
          [2., 2., 3., 3., 5., 5., 7., 7.],
          [2., 2., 3., 3., 5., 5., 7., 7.]]]])

In [84]:
mask_pos = (mask>0).type(torch.float)
mask_pos

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [58]:
diag = torch.eye(len(y))
diag

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])

In [70]:
# targets 2N elements.
targets = torch.arange(4)
print(targets)
targets[::2] += 1
print(targets) 
targets[1::2] -= 1
print(targets)

tensor([0, 1, 2, 3])
tensor([1, 1, 3, 3])
tensor([1, 0, 3, 2])


In [25]:
import torchvision

d_train = torchvision.datasets.CIFAR10('data/cifar10', train=True, transform=None, target_transform=None, download=True)
d_test = torchvision.datasets.CIFAR10('data/cifar10', train=False, transform=None, target_transform=None, download=True)

100%|██████████| 170498071/170498071 [00:59<00:00, 2853146.35it/s]


Extracting data/cifar10/cifar-10-python.tar.gz to data/cifar10
Files already downloaded and verified


In [28]:
import pickle as p

p_train, p_test = [], []

for img,target in zip(d_train.data,d_train.targets):
    l = [img,int(target)]
    p_train.append(l)

for img,target in zip(d_test.data, d_test.targets):
    l = [img,int(target)]
    p_test.append(l)

p.dump(p_train,open('./tmp/cifar10/train_combined.pkl','wb'))
p.dump(p_test,open('./tmp/cifar10/test_combined.pkl','wb'))

FileNotFoundError: [Errno 2] No such file or directory: './tmp/cifar10/train_combined.pkl'

In [176]:
x = torch.randint(0, 4, (4, 2)) # (N, C)
print(x)
cov = np.cov(x, rowvar= False) # (N, N)
print(cov)

tensor([[2, 1],
        [2, 2],
        [2, 3],
        [3, 0]])
[[ 0.25       -0.5       ]
 [-0.5         1.66666667]]


In [196]:
x = torch.randn((4, 2)) # (B, C)
print(x)
print(x.max(1)[1])
x = x.view(x.size(0), x.size(1), -1)
print(x.shape)

tensor([[ 1.2132, -0.2886],
        [ 2.0017, -0.8415],
        [ 0.1093,  1.2570],
        [ 0.7835,  1.8634]])
tensor([0, 0, 1, 1])
torch.Size([4, 2, 1])


### Mahalanobis

In [268]:
# load networks
from resnet import ResNet34
model = ResNet34(num_c=10)

# set information about feature extaction
model.eval()
temp_x = torch.rand(32, 3, 32, 32)
temp_x = temp_x.detach()
temp_list = model.feature_list(temp_x)[1] # (B, C, H, W)
num_output = len(temp_list)
print("num of layers: ", num_output)
for i in range(len(temp_list)):
    print("size of each layer: ", temp_list[i].shape)

feature_list = np.empty(num_output) # [0] * num_layers
count = 0
for out in temp_list:
    feature_list[count] = out.size(1) # C
    count += 1
print("feature_list:", feature_list)

num of layers:  5
size of each layer:  torch.Size([32, 64, 32, 32])
size of each layer:  torch.Size([32, 64, 32, 32])
size of each layer:  torch.Size([32, 128, 16, 16])
size of each layer:  torch.Size([32, 256, 8, 8])
size of each layer:  torch.Size([32, 512, 4, 4])
feature_list: [ 64.  64. 128. 256. 512.]


In [269]:
out_features = temp_list
for i in range(num_output):
    out_features[i] = out_features[i].view(out_features[i].size(0), out_features[i].size(1), -1) # (B, C, H*W)
    out_features[i] = torch.mean(out_features[i].data, 2) # (B, C)
    print(out_features[i].data.shape)

torch.Size([32, 64])
torch.Size([32, 64])
torch.Size([32, 128])
torch.Size([32, 256])
torch.Size([32, 512])


In [271]:
num_classes = 10
num_sample_per_class = np.empty(num_classes) # [0] * 10
num_sample_per_class.fill(0)
list_features = []
for i in range(num_output):
    temp_list = [] # [0] * 10
    for j in range(num_classes):
        temp_list.append(0)
    list_features.append(temp_list) # [[0] * 10] * num_layers

target = torch.randint(0, 10, (32, ))
data = temp_x

# construct the sample matrix
for i in range(data.size(0)): # [0, B-1]
    label = target[i]
    if num_sample_per_class[label] == 0:
        out_count = 0
        for out in out_features: # out: (B, num_feature)
            list_features[out_count][label] = out[i].view(1, -1) # (1, num_feature)
            out_count += 1
    else:
        out_count = 0
        for out in out_features:
            list_features[out_count][label] \
            = torch.cat((list_features[out_count][label], out[i].view(1, -1)), 0)
            out_count += 1                
    num_sample_per_class[label] += 1

In [275]:
# (num_layers, num_class, N_c, C)
print(len(list_features))
print(len(list_features[0]))
tmp = 0
for i in range(num_classes):
    tmp += len(list_features[0][i])
print(tmp)
print(len(list_features[0][0][0]))

5
10
32
64


In [286]:
sample_class_mean = [] # [(num_classes, C)] * num_layers
out_count = 0
for num_feature in feature_list: # num_layers
    temp_list = torch.Tensor(num_classes, int(num_feature)) # (num_classes, C)
    for j in range(num_classes):
        temp_list[j] = torch.mean(list_features[out_count][j], 0) # (N_c, C) -> (1, C)
    sample_class_mean.append(temp_list)
    out_count += 1
print(len(sample_class_mean))
print(sample_class_mean[0].shape)

5
torch.Size([10, 64])


In [293]:
import sklearn.covariance

group_lasso = sklearn.covariance.EmpiricalCovariance(assume_centered=False)

precision = []
for k in range(num_output):
    X = 0
    for i in range(num_classes):
        if i == 0:
            X = list_features[k][i] - sample_class_mean[k][i] # (N_c, C)
        else:
            X = torch.cat((X, list_features[k][i] - sample_class_mean[k][i]), 0) # (N, C)
    # find inverse            
    group_lasso.fit(X.numpy())
    temp_precision = group_lasso.precision_
    temp_precision = torch.from_numpy(temp_precision).float() # (C, N)
    precision.append(temp_precision) # [(C, N)] * num_layers
print(len(precision))
print(precision[0].shape) # [(C, C)] * num_layers

5
torch.Size([64, 64])


In [345]:
x = torch.randn((3, 2)) # (B, C)
sigma = torch.randn((2, 2)) # (C, C)
print(x)
print(sigma)

tensor([[ 0.4317,  0.1070],
        [-0.3850, -0.6187],
        [ 0.4838,  0.0031]])
tensor([[-2.2033,  2.2806],
        [ 0.0719,  1.6152]])


In [353]:
y = torch.mm(torch.mm(x, sigma), x.t()).diag()
y = y.view(-1, 1)
print(y.shape)

torch.Size([3, 1])


In [358]:
t = torch.randn((3, 1))
t = torch.cat((y, t), 1)
print(t)
print(t.shape)

tensor([[-0.2834,  0.0621],
        [ 0.8519, -0.5122],
        [-0.5122,  0.8572]])
torch.Size([3, 2])


In [361]:
t.max(1)

torch.return_types.max(
values=tensor([0.0621, 0.8519, 0.8572]),
indices=tensor([1, 0, 1]))

In [374]:
x = torch.randn((4, 2))
y = torch.randn((4, 2))
z = torch.sum(x * y, dim=1)
print(z.shape)
z

torch.Size([4])


tensor([ 0.1002, -1.0799,  0.3467,  0.8103])

In [376]:
for i in range(4):
    print(sum(x[i]*y[i]))    

tensor(0.1002)
tensor(-1.0799)
tensor(0.3467)
tensor(0.8103)


In [377]:
torch.mean(z)

tensor(0.0443)

In [29]:
import torch

x = torch.randn((4, 1))
x

tensor([[-2.1411],
        [ 1.1586],
        [-0.7949],
        [ 1.0995]])

In [45]:
x = torch.zeros(10, dtype=torch.long)
x.shape

torch.Size([10])

In [46]:
index = [0, 1, 2, 4, 6]
p = x[index]
p.shape

torch.Size([5])

In [36]:
torch.linspace(0, x.size(0)-1, steps=x.size(0)).long()

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [53]:
r = torch.zeros(4, dtype=torch.long)

In [54]:
y = torch.cat([p, r],dim=0)
y.shape

torch.Size([9])

In [55]:
t = torch.zeros(500)
t[y]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [24]:
from collections import Counter
from typing import List

def minimumSwap(s1: str, s2: str) -> int:
    ans = 0
    c1 = Counter(s1)
    c2 = Counter(s2)

    x = c1["x"]+c2["x"]
    y = c1["y"]+c2["y"]
    print(x, y)
    if x % 2 != 0 or y % 2 != 0:
        return -1

    tmp = []
    n = len(s1)
    for i in range(n):
        if s1[i] != s2[i]:
            tmp.append(s1[i])
    
    for j in range(len(tmp)-1):
        if tmp[j] == tmp[j+1]:
            ans += 1
        else:
            ans += 2
    return ans

s1 = "xxyyxyxyxx"
s2 = "xyyxyxxxyx"
minimumSwap(s1, s2)

12 8


9